# Canine Colonocyte Butyrate Metabolism: Model Caninization Workflow

**Objective**: Convert human colonocyte butyrate metabolism model (Human-GEM) to canine ortholog-based model.

**Approach**:
1. Load Human-GEM SBML model
2. Filter butyrate pathway reactions
3. Map human → dog gene orthologs
4. Substitute GPR (Gene-Protein-Reaction) associations
5. Validate model integrity
6. Apply physiological bounds
7. Run FBA/pFBA analysis
8. Visualize and export results

---

## Block 1: Setup & Imports

In [4]:
# Core libraries
import cobra
from cobra.flux_analysis import pfba, flux_variability_analysis
import pandas as pd
import numpy as np
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"COBRApy version: {cobra.__version__}")
print(f"Pandas version: {pd.__version__}")
print("\n✓ All imports successful")

COBRApy version: 0.30.0
Pandas version: 2.3.3

✓ All imports successful


## Block 2: Load Human-GEM Model

**Model source**: Human-GEM (Recon3D) - download from [GitHub](https://github.com/SysBioChalmers/Human-GEM)

**Expected**: ~13,400 reactions, ~4,100 genes

In [5]:
# Load Human-GEM model
MODEL_PATH = "../data/Human-GEM.xml"

try:
    model_human = cobra.io.read_sbml_model(MODEL_PATH)
    print(f"✓ Model loaded: {model_human.id}")
    print(f"  Reactions: {len(model_human.reactions)}")
    print(f"  Metabolites: {len(model_human.metabolites)}")
    print(f"  Genes: {len(model_human.genes)}")
except FileNotFoundError:
    print(f"⚠ Model file not found at {MODEL_PATH}")
    print("  Download Human-GEM.xml from GitHub and place in data/ folder")
    model_human = None

✓ Model loaded: HumanGEM
  Reactions: 12971
  Metabolites: 8455
  Genes: 2887


### 👀 Preview: Sample Reactions, Metabolites, Genes

In [6]:
if model_human:
    print("=== SAMPLE REACTIONS (first 5) ===")
    for rxn in list(model_human.reactions)[:5]:
        print(f"\nID: {rxn.id}")
        print(f"Name: {rxn.name}")
        print(f"Equation: {rxn.reaction}")
        print(f"GPR: {rxn.gene_reaction_rule or 'None'}")
        print(f"Bounds: [{rxn.lower_bound}, {rxn.upper_bound}]")
    
    print("\n" + "="*60)
    print("=== SAMPLE METABOLITES (first 5) ===")
    for met in list(model_human.metabolites)[:5]:
        print(f"\nID: {met.id}")
        print(f"Name: {met.name}")
        print(f"Compartment: {met.compartment}")
        print(f"Formula: {met.formula or 'N/A'}")
    
    print("\n" + "="*60)
    print("=== SAMPLE GENES (first 10) ===")
    gene_list = list(model_human.genes)[:10]
    for gene in gene_list:
        rxn_count = len(gene.reactions)
        print(f"  {gene.id:15s} → {rxn_count:3d} reactions")

=== SAMPLE REACTIONS (first 5) ===

ID: MAR03905
Name: ethanol:NAD+ oxidoreductase
Equation: MAM01796c + MAM02552c --> MAM01249c + MAM02039c + MAM02553c
GPR: ENSG00000147576 or ENSG00000172955 or ENSG00000180011 or ENSG00000187758 or ENSG00000196344 or ENSG00000196616 or ENSG00000197894 or ENSG00000198099 or ENSG00000248144
Bounds: [0.0, 1000.0]

ID: MAR03907
Name: Ethanol:NADP+ oxidoreductase
Equation: MAM01796c + MAM02554c --> MAM01249c + MAM02039c + MAM02555c
GPR: ENSG00000117448
Bounds: [0.0, 1000.0]

ID: MAR04097
Name: Acetate:CoA ligase (AMP-forming)
Equation: MAM01252c + MAM01371c + MAM01597c --> MAM01261c + MAM01334c + MAM02759c
GPR: ENSG00000131069
Bounds: [0.0, 1000.0]

ID: MAR04099
Name: Acetate:CoA ligase (AMP-forming)
Equation: MAM01252m + MAM01371m + MAM01597m --> MAM01261m + MAM01334m + MAM02759m
GPR: ENSG00000111058 or ENSG00000154930
Bounds: [0.0, 1000.0]

ID: MAR04108
Name: acetyl adenylate:CoA acetyltransferase
Equation: MAM01257c + MAM01597c --> MAM01261c + MAM01334

## Block 3: Filter Butyrate Pathway Reactions

**Target subsystems**:
- SCFA transport (lumen → cytosol)
- Butyrate activation (→ butyryl-CoA)
- β-oxidation (C4 pathway)
- Electron transfer (ETFA/B/DH)
- OXPHOS (Complexes I-V)

In [ ]:
# ============================================
# OPTION 1: MANUAL OVERRIDE (Researcher-provided reaction IDs)
# ============================================
# If the researcher provides a curated list of reaction IDs, set them here:
MANUAL_REACTION_IDS = []  # Example: ['MAR03163', 'MAR04099', 'MAR07709']

# ============================================
# OPTION 2: AUTOMATIC KEYWORD SEARCH
# ============================================
if model_human:
    if MANUAL_REACTION_IDS:
        # Use manual list
        print("=== USING MANUAL REACTION LIST ===")
        but_reactions = []
        for rxn_id in MANUAL_REACTION_IDS:
            if rxn_id in model_human.reactions:
                but_reactions.append(model_human.reactions.get_by_id(rxn_id))
                print(f"  ✓ {rxn_id}")
            else:
                print(f"  ⚠ {rxn_id} NOT FOUND in model")
        
        print(f"\n✓ Loaded {len(but_reactions)} reactions from manual list")
    
    else:
        # Automatic keyword search with improved specificity
        print("=== AUTOMATIC KEYWORD SEARCH ===")
        
        # More specific keywords to avoid false positives
        include_keywords = [
            'butyr',          # butyrate, butyryl, butyric
            'butanoyl-coa',   # butanoyl-CoA (specific to butyrate metabolism)
            'c4:0',           # C4 notation for butyrate
            'mct1', 'mct4',   # Monocarboxylate transporters
            'smct',           # Sodium-coupled MCT
            'slc16a1',        # MCT1 gene
            'slc5a8', 'slc5a12'  # SMCT genes
        ]
        
        # Keywords to exclude (false positives)
        exclude_keywords = [
            'aminobut',       # aminobutanoate, aminobutyrate (GABA pathway)
            'ureidobut',      # ureidoisobutyrate (pyrimidine degradation)
            'methylbut',      # methylbutanoyl (leucine metabolism)
            'hydroxybut',     # hydroxybutyrate (ketone bodies, not butyrate)
        ]
        
        but_reactions = []
        excluded_count = 0
        
        for rxn in model_human.reactions:
            rxn_str = (rxn.id + " " + rxn.name).lower()
            
            # Check if matches include keywords
            has_include = any(kw in rxn_str for kw in include_keywords)
            
            # Check if matches exclude keywords
            has_exclude = any(kw in rxn_str for kw in exclude_keywords)
            
            if has_include and not has_exclude:
                but_reactions.append(rxn)
            elif has_include and has_exclude:
                excluded_count += 1
        
        print(f"Found {len(but_reactions)} butyrate-related reactions")
        print(f"Excluded {excluded_count} false positives (aminobutanoate, etc.)")
        
        print("\nSample reactions (first 10):")
        for rxn in but_reactions[:10]:
            print(f"  {rxn.id:30s} {rxn.name}")
    
    # Extract unique genes from these reactions
    but_genes = set()
    for rxn in but_reactions:
        but_genes.update(rxn.genes)
    
    print(f"\n✓ {len(but_genes)} genes associated with butyrate reactions")
    if but_genes:
        print("\nSample genes (first 10):")
        for gene in list(but_genes)[:10]:
            print(f"  - {gene.id}")
else:
    print("⚠ Skipping - model not loaded")

### Create Butyrate Sub-Model

Extract core pathway + essential reactions:
- Butyrate reactions (filtered above)
- OXPHOS (ATP synthase, respiratory chain)
- Exchange reactions (EX_but, EX_o2, EX_co2, etc.)
- ATPM (ATP maintenance)

In [8]:
if model_human:
    # Core genes to include (from ortholog mapping file)
    core_genes = [
        'SLC16A1',  # MCT1
        'SLC5A8', 'SLC5A12',  # SMCT1/2
        'ACSM2A', 'ACSM2B', 'ACSS3',  # Activation
        'ACADS',  # β-oxidation
        'ECHS1', 'HADHA', 'HADHB', 'ACAT1',  # β-oxidation
        'ETFA', 'ETFB', 'ETFDH'  # Electron transfer
    ]
    
    print("=== Finding reactions for 14 core genes ===")
    # Find reactions associated with core genes
    core_reactions = set()
    for gene_id in core_genes:
        gene = model_human.genes.get_by_id(gene_id) if gene_id in model_human.genes else None
        if gene:
            core_reactions.update(gene.reactions)
            print(f"  {gene_id:12s} → {len(gene.reactions):3d} reactions")
        else:
            print(f"  {gene_id:12s} → NOT FOUND")
    
    print(f"\n✓ Total core reactions: {len(core_reactions)}")
    
    # Add essential subsystems
    print("\n=== Adding OXPHOS & respiratory chain ===")
    essential_keywords = ['oxphos', 'respiratory', 'atp synthase', 'atpm', 'complex']
    oxphos_count = 0
    for rxn in model_human.reactions:
        rxn_str = (rxn.id + " " + rxn.name + " " + (rxn.subsystem or "")).lower()
        if any(kw in rxn_str for kw in essential_keywords):
            if rxn not in core_reactions:
                oxphos_count += 1
            core_reactions.add(rxn)
    print(f"  Added {oxphos_count} OXPHOS reactions")
    
    # Add exchange reactions
    print("\n=== Adding exchange reactions ===")
    exchange_count = 0
    for rxn in model_human.exchanges:
        if any(met in rxn.id.lower() for met in ['but', 'o2', 'co2', 'h2o', 'h_', 'pi']):
            if rxn not in core_reactions:
                exchange_count += 1
                print(f"  + {rxn.id}")
            core_reactions.add(rxn)
    print(f"  Added {exchange_count} exchange reactions")
    
    print(f"\n✓ TOTAL CORE REACTIONS: {len(core_reactions)}")
else:
    print("⚠ Skipping - model not loaded")

=== Finding reactions for 14 core genes ===
  SLC16A1      → NOT FOUND
  SLC5A8       → NOT FOUND
  SLC5A12      → NOT FOUND
  ACSM2A       → NOT FOUND
  ACSM2B       → NOT FOUND
  ACSS3        → NOT FOUND
  ACADS        → NOT FOUND
  ECHS1        → NOT FOUND
  HADHA        → NOT FOUND
  HADHB        → NOT FOUND
  ACAT1        → NOT FOUND
  ETFA         → NOT FOUND
  ETFB         → NOT FOUND
  ETFDH        → NOT FOUND

✓ Total core reactions: 0

=== Adding OXPHOS & respiratory chain ===
  Added 12 OXPHOS reactions

=== Adding exchange reactions ===
  Added 0 exchange reactions

✓ TOTAL CORE REACTIONS: 12


### 👀 Preview: Sample Core Reactions with GPRs

In [ ]:
if model_human and core_reactions:
    print("=== SAMPLE CORE REACTIONS (showing GPR logic) ===")
    sample_rxns = list(core_reactions)[:8]
    for rxn in sample_rxns:
        print(f"\nReaction: {rxn.id}")
        print(f"  Name: {rxn.name}")
        print(f"  Equation: {rxn.reaction}")
        print(f"  GPR: {rxn.gene_reaction_rule or 'None'}")
        if rxn.genes:
            gene_ids = [g.id for g in rxn.genes]
            print(f"  Genes: {', '.join(gene_ids)}")

## Block 4: Load Ortholog Mappings

Load human → dog gene mappings from curated Excel file.

In [ ]:
# Load ortholog mapping file
ORTHOLOG_FILE = "../data/02_human_dog_orthologs.xlsx"

try:
    df_orthologs = pd.read_excel(ORTHOLOG_FILE)
    print(f"✓ Loaded {len(df_orthologs)} ortholog mappings\n")
    print("Columns:", list(df_orthologs.columns))
    print("\n=== ORTHOLOG MAPPINGS TABLE ===")
    display(df_orthologs.head(15))  # Show all 14 + header
    
    # Create mapping dictionary: human_gene → dog_gene
    ortholog_map = dict(zip(
        df_orthologs['Gene umano'],
        df_orthologs['Ortologo canino']
    ))
    
    print(f"\n✓ Created mapping dictionary for {len(ortholog_map)} genes")
    print("\n=== MAPPING DICTIONARY (sample) ===")
    for i, (human, dog) in enumerate(list(ortholog_map.items())[:5]):
        print(f"  {human:12s} → {dog}")
    
except FileNotFoundError:
    print(f"⚠ File not found: {ORTHOLOG_FILE}")
    ortholog_map = {}
except Exception as e:
    print(f"⚠ Error loading file: {e}")
    print("  Check column names and file format")
    ortholog_map = {}

## Block 5: Substitute GPRs (Human → Dog)

Replace human gene IDs with canine orthologs in Gene-Protein-Reaction rules.

**GPR Logic**:
- `AND`: protein complex (all genes required)
- `OR`: isozymes (any gene sufficient)

In [ ]:
if model_human and ortholog_map:
    # Create canine model (copy)
    model_canine = model_human.copy()
    model_canine.id = "CanineColon_Butyrate"
    
    # Track substitutions
    substituted = 0
    not_found = set()
    examples_before_after = []  # Store examples for preview
    
    print("=== Substituting Human → Dog genes in GPRs ===")
    
    for rxn in model_canine.reactions:
        gpr_original = rxn.gene_reaction_rule
        if not gpr_original:
            continue
        
        # Extract human gene IDs from GPR
        gene_ids = [g.id for g in rxn.genes]
        
        # Substitute genes
        gpr_canine = gpr_original
        has_substitution = False
        for human_gene in gene_ids:
            if human_gene in ortholog_map:
                dog_gene = ortholog_map[human_gene]
                # Replace gene ID in GPR string
                gpr_canine = re.sub(
                    r'\b' + re.escape(human_gene) + r'\b',
                    dog_gene,
                    gpr_canine
                )
                substituted += 1
                has_substitution = True
            else:
                not_found.add(human_gene)
        
        # Update GPR
        if gpr_canine != gpr_original:
            # Store example for preview
            if len(examples_before_after) < 5 and has_substitution:
                examples_before_after.append({
                    'reaction': rxn.id,
                    'before': gpr_original,
                    'after': gpr_canine
                })
            rxn.gene_reaction_rule = gpr_canine
    
    print(f"\n✓ Substituted {substituted} gene occurrences")
    print(f"  Genes without mapping: {len(not_found)}")
    if not_found:
        print("\nUnmapped genes (sample, first 10):")
        for gene in list(not_found)[:10]:
            print(f"  - {gene}")
    
    print(f"\n✓ Canine model created: {model_canine.id}")
    print(f"  Reactions: {len(model_canine.reactions)}")
    print(f"  Genes: {len(model_canine.genes)}")
else:
    print("⚠ Skipping - prerequisites not met")
    model_canine = None

### 👀 Preview: GPR Before/After Substitution

In [ ]:
if model_canine and examples_before_after:
    print("=== GPR SUBSTITUTION EXAMPLES ===")
    for i, ex in enumerate(examples_before_after, 1):
        print(f"\nExample {i}: {ex['reaction']}")
        print(f"  BEFORE (Human): {ex['before']}")
        print(f"  AFTER  (Dog):   {ex['after']}")

## Block 6: Validate Model Integrity

**Checks**:
1. GPR integrity (no broken parentheses)
2. Orphan reactions (reactions without genes in critical pathways)
3. Mass/charge balance
4. Core pathway completeness

In [ ]:
if model_canine:
    print("="*60)
    print("=== MODEL VALIDATION ===")
    print("="*60)
    
    # 1. Check for reactions without GPR (orphans)
    orphan_reactions = [r for r in model_canine.reactions if not r.genes]
    print(f"\n1. Orphan reactions (no GPR): {len(orphan_reactions)}")
    
    # Check core pathway orphans
    core_orphans = [r for r in core_reactions if not r.genes]
    if core_orphans:
        print(f"   ⚠ {len(core_orphans)} orphans in CORE pathway:")
        for rxn in core_orphans[:5]:
            print(f"     - {rxn.id}: {rxn.name}")
    else:
        print("   ✓ No orphans in core butyrate pathway")
    
    # 2. Check GPR syntax (balanced parentheses)
    broken_gpr = []
    for rxn in model_canine.reactions:
        gpr = rxn.gene_reaction_rule
        if gpr and gpr.count('(') != gpr.count(')'):
            broken_gpr.append(rxn.id)
    
    print(f"\n2. GPR syntax errors: {len(broken_gpr)}")
    if broken_gpr:
        print(f"   ⚠ Broken GPRs (first 5): {broken_gpr[:5]}")
    else:
        print("   ✓ All GPRs syntactically valid")
    
    # 3. Check core genes presence
    print("\n3. Core gene verification (14 genes):")
    print(f"{'Human Gene':<15} {'Dog Ortholog':<25} {'Present'}")
    print("-" * 50)
    for gene_id in core_genes:
        dog_gene = ortholog_map.get(gene_id, "NOT_MAPPED")
        present = dog_gene in model_canine.genes
        status = "✓" if present else "✗"
        print(f"{gene_id:<15} {dog_gene:<25} {status}")
    
    print("\n" + "="*60)
    print("✓ VALIDATION COMPLETE")
    print("="*60)
else:
    print("⚠ Skipping - model not available")

## Block 7: Apply Physiological Bounds & Run FBA

Apply "healthy dog" baseline constraints and test metabolic flux.

In [ ]:
# Load physiological bounds
BOUNDS_FILE = "../data/01_physiological_bounds.xlsx"

try:
    df_bounds = pd.read_excel(BOUNDS_FILE)
    print("✓ Loaded physiological bounds\n")
    print("=== PHYSIOLOGICAL BOUNDS TABLE ===")
    display(df_bounds)
except FileNotFoundError:
    print(f"⚠ File not found: {BOUNDS_FILE}")
    df_bounds = None

In [ ]:
if model_canine and df_bounds is not None:
    print("="*60)
    print("=== APPLYING BOUNDS ===")
    print("="*60)
    
    # Apply bounds from dataframe
    for _, row in df_bounds.iterrows():
        rxn_id = row['Reaction_ID']
        lb = row['Lower Bound (mmol/gDW/h)']
        ub = row['Upper Bound (mmol/gDW/h)']
        
        if rxn_id in model_canine.reactions:
            rxn = model_canine.reactions.get_by_id(rxn_id)
            rxn.lower_bound = float(lb)
            rxn.upper_bound = float(ub)
            met_name = row.get('Metabolite', 'N/A')
            print(f"  {rxn_id:15s} [{lb:7.1f}, {ub:7.1f}]  # {met_name}")
        else:
            print(f"  ⚠ Reaction not found: {rxn_id}")
    
    # Set objective: ATP maintenance (ATPM)
    if 'ATPM' in model_canine.reactions:
        model_canine.objective = 'ATPM'
        print("\n✓ Objective set: ATPM (ATP maintenance)")
    else:
        # Search for alternative
        atpm_candidates = [r.id for r in model_canine.reactions if 'atpm' in r.id.lower()]
        if atpm_candidates:
            model_canine.objective = atpm_candidates[0]
            print(f"\n✓ Objective set: {atpm_candidates[0]}")
        else:
            print("\n⚠ ATPM reaction not found")
    
    print("\n" + "="*60)
    print("=== RUNNING FBA (Flux Balance Analysis) ===")
    print("="*60)
    solution = model_canine.optimize()
    
    print(f"\nStatus: {solution.status}")
    print(f"Objective value (ATPM): {solution.objective_value:.6f} mmol/gDW/h")
    
    # Display key fluxes
    print("\n=== KEY METABOLIC FLUXES ===")
    print(f"{'Reaction ID':<20} {'Flux (mmol/gDW/h)':<20} {'Interpretation'}")
    print("-" * 70)
    
    flux_interpretations = {
        'EX_but_lum': 'Butyrate uptake from lumen',
        'EX_ac_lum': 'Acetate uptake from lumen',
        'EX_pro_lum': 'Propionate uptake from lumen',
        'EX_o2_bld': 'O₂ consumption (hypoxia)',
        'EX_co2_bld': 'CO₂ production',
        'ATPM': 'ATP produced (energy output)'
    }
    
    for rxn_id, description in flux_interpretations.items():
        if rxn_id in model_canine.reactions:
            flux = solution.fluxes.get(rxn_id, 0.0)
            print(f"{rxn_id:<20} {flux:>10.6f}          {description}")
    
    # Run pFBA (parsimonious FBA)
    print("\n" + "="*60)
    print("=== RUNNING pFBA (Parsimonious FBA) ===")
    print("="*60)
    pfba_solution = pfba(model_canine)
    print(f"\npFBA objective: {pfba_solution.objective_value:.6f}")
    print(f"Total flux (sum of all reactions): {pfba_solution.fluxes.abs().sum():.2f}")
    print("\nNote: pFBA minimizes total flux while maintaining optimal objective.")
    print("This gives more realistic flux distributions.")
    
else:
    print("⚠ Skipping - prerequisites not met")

### 👀 Preview: Top Active Reactions

In [ ]:
if model_canine and solution.status == 'optimal':
    print("=== TOP 10 ACTIVE REACTIONS (by absolute flux) ===")
    print(f"\n{'Rank':<6} {'Reaction ID':<20} {'Flux':<15} {'Reaction Name'}")
    print("-" * 80)
    
    # Get top reactions by absolute flux
    flux_abs = solution.fluxes.abs().sort_values(ascending=False)
    top_reactions = flux_abs.head(10)
    
    for rank, (rxn_id, abs_flux) in enumerate(top_reactions.items(), 1):
        if rxn_id in model_canine.reactions:
            rxn = model_canine.reactions.get_by_id(rxn_id)
            flux_value = solution.fluxes[rxn_id]
            print(f"{rank:<6} {rxn_id:<20} {flux_value:>10.6f}     {rxn.name[:40]}")

## Block 8: Results Visualization & Export

Visualize flux distributions and export canine model.

In [ ]:
if model_canine and solution.status == 'optimal':
    # Plot key fluxes
    key_rxns = ['EX_but_lum', 'EX_ac_lum', 'EX_pro_lum', 'EX_o2_bld']
    fluxes = [solution.fluxes.get(r, 0) for r in key_rxns]
    labels = ['Butyrate\nuptake', 'Acetate\nuptake', 'Propionate\nuptake', 'O₂\nconsumption']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(labels, fluxes, color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
    ax.set_xlabel('Flux (mmol/gDW/h)', fontsize=12)
    ax.set_title('Key Metabolite Exchange Fluxes - Canine Colon Model', fontsize=14, fontweight='bold')
    ax.axvline(0, color='black', linewidth=0.8)
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for bar, flux in zip(bars, fluxes):
        width = bar.get_width()
        label_x_pos = width + (0.5 if width < 0 else -0.5)
        ax.text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{flux:.2f}',
                ha='left' if width < 0 else 'right', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Export canine model
    print("\n" + "="*60)
    print("=== EXPORTING RESULTS ===")
    print("="*60)
    
    output_path = "../data/CanineColon_Butyrate.xml"
    cobra.io.write_sbml_model(model_canine, output_path)
    print(f"\n✓ Canine model exported: {output_path}")
    
    # Export flux results
    flux_df = pd.DataFrame({
        'Reaction': solution.fluxes.index,
        'Flux_FBA': solution.fluxes.values,
        'Flux_pFBA': pfba_solution.fluxes.values
    })
    # Filter near-zero fluxes
    flux_df = flux_df[flux_df['Flux_FBA'].abs() > 1e-6]
    flux_df = flux_df.sort_values('Flux_FBA', key=abs, ascending=False)
    
    flux_csv_path = "../data/canine_flux_results.csv"
    flux_df.to_csv(flux_csv_path, index=False)
    print(f"✓ Flux results exported: {flux_csv_path}")
    
    print("\n=== FLUX RESULTS PREVIEW (top 10) ===")
    display(flux_df.head(10))
    
    print("\n" + "="*60)
    print("✓ CANINIZATION WORKFLOW COMPLETE")
    print("="*60)
    print(f"\nSummary:")
    print(f"  • Human-GEM reactions: {len(model_human.reactions)}")
    print(f"  • Core reactions extracted: {len(core_reactions)}")
    print(f"  • Genes substituted: {len(ortholog_map)}")
    print(f"  • Canine model reactions: {len(model_canine.reactions)}")
    print(f"  • ATP production (ATPM): {solution.objective_value:.4f} mmol/gDW/h")
    print(f"  • Active reactions (|flux| > 1e-6): {len(flux_df)}")
    
else:
    print("⚠ Cannot visualize - optimization failed or model unavailable")

---

## Next Steps

1. **Validate with experimental data**: Compare predicted fluxes with measured SCFA concentrations
2. **Sensitivity analysis**: Test robustness to parameter variations
3. **Dysbiosis scenarios**: Simulate reduced butyrate availability
4. **Expand model**: Add glycolysis, TCA cycle, amino acid metabolism
5. **Microbiota integration**: Couple with microbial community models

---

**Version**: 1.1 (with data previews)  
**Date**: 2025-01-12  
**Documentation**: See `docs/CONCEPTS_EXPLAINED.md` for detailed explanations